In [1]:
import os
import datetime
import matplotlib.pylab as plt
import numpy as np
import pandas as pd
import glob
import scipy.io as sio

import netCDF4 as nc
from salishsea_tools import tidetools
from nowcast import (analyze, figures, research_ferries_test as rft)

%matplotlib inline

In [20]:
from importlib import reload

In [10]:
today = datetime.datetime.today()
yesterday = today - datetime.timedelta(days=12)
tomorrow = today + datetime.timedelta(days=1)
period = '1h'
grid = 'grid_T'
dmy = '07dec15'
date = datetime.datetime.strptime(dmy, "%d%b%y")
dmyf = '06dec15'
datef = datetime.datetime.strptime(dmyf, "%d%b%y")
print(date)

2015-12-07 00:00:00


Step 0: Load the datasets and files that are available in nowcast/make_plots. 
These will be used as inputs, not to be reloaded in the module

In [34]:
grid_T_hr = nc.Dataset(analyze.get_filenames(yesterday, tomorrow, period, grid, '/results/SalishSea/nowcast/')[0])
coastline = sio.loadmat('/ocean/rich/more/mmapbase/bcgeo/PNW.mat')
bathy, X, Y = tidetools.get_SS2_bathy_data()

## A : Preparing ferry data
Step 1: Get ferry data

In [35]:
reload(rft)

<module 'nowcast.research_ferries_test' from '/ocean/mdunn/MEOPAR/tools/SalishSeaNowcast/nowcast/research_ferries_test.py'>

In [11]:
obs = rft._get_sal_data('TWDP', dmyf)

Step 2: Slicing ferry data

In [12]:
route_name='TWDP'
start_time = date.replace(hour=rft.route[route_name]['start']['hour'], minute=rft.route[route_name]['start']['minute'])
end_time = date.replace(hour=rft.route[route_name]['end']['hour'], minute=rft.route[route_name]['end']['minute'])
print(start_time)

2015-12-07 03:00:00


In [49]:
obs_sal = rft.ferry_salinity('TWDP', dmyf, dmy)
print(obs_sal[1])

[-123.44653029 -123.42497914 -123.40323453 -123.38139788 -123.35941705
 -123.33458341 -123.3092457  -123.28389498 -123.25834522 -123.23297168
 -123.20718208 -123.18179465 -123.1568823  -123.13731329]


## B: Preparing model data

In [42]:
sal3, sal4 = rft.nemo_salinity('TWDP', grid_T_hr)

2.12874e+06


## C: Find corresponding model point to obs data

In [51]:
tidetools.find_closest_model_point(obs_sal[1,1], obs_sal[2,1], X, Y, bathy, lat_tol=0.00210)

(429, 269)